In [1]:
import pybedtools
import numpy as np
import pysam
from collections import defaultdict
import pickle
import multiprocessing as mp

In [2]:
path = '/home/matvey/Документы/Data_Analysis_in_Biology_and_Medicine/new_science/data'

In [3]:
genes = pybedtools.BedTool(f'{path}/genes_by_chrom.bed')# предварительно созданный файл по генам с 24 хромосом

In [4]:
genes.head()

chr1	11868	14409	+
 chr1	14403	29570	-
 chr1	29553	31109	+
 chr1	34553	36081	-
 chr1	52472	53312	+
 chr1	57597	64116	+
 chr1	65418	71585	+
 chr1	89294	133723	-
 chr1	131024	134836	+
 chr1	135140	135895	-
 

{chrom:[координаты А в генах, T в негенах и A/T ]}

In [5]:
count_teor_sites = defaultdict(int)
for chrom in genes:
    count_teor_sites[chrom.chrom] += 1

In [6]:
count_teor_sites

defaultdict(int,
            {'chr1': 4624,
             'chr2': 3526,
             'chr3': 2567,
             'chr4': 2266,
             'chr5': 2485,
             'chr6': 2613,
             'chr7': 2482,
             'chr8': 2095,
             'chr9': 1949,
             'chrX': 2069,
             'chrY': 505,
             'chr10': 1971,
             'chr11': 2757,
             'chr12': 2409,
             'chr13': 1149,
             'chr14': 1863,
             'chr15': 1620,
             'chr16': 1989,
             'chr17': 2367,
             'chr18': 990,
             'chr19': 2389,
             'chr20': 1207,
             'chr21': 728,
             'chr22': 1132})

In [7]:
count_teor_sites['chr1']

4624

In [8]:
sum(count_teor_sites.values())

49752

In [9]:
def gene_search(chrom: str, start: int, end: int) -> str:
    """Функция принимает на вход хромосому и координаты в ней. Выводит подстроку генома"""
    with pysam.FastaFile(f'{path}/GRCh38_24.fasta') as fh:
        gene_str = fh.fetch(chrom, start, end)
    return gene_str

### Вывод результатов в один файл

teor_sites = defaultdict(list)
for j, gene in enumerate(genes):
    chr_list = []
    gene_by_letter = gene_search(gene[0], int(gene[1]), int(gene[2]))
    for i, letter in enumerate(gene_by_letter):
        if gene[3] == '+' and letter == 'A':
            chr_list.append(int(gene[1])+i)
        if gene[3] == '-' and letter == 'T':
            chr_list.append(int(gene[1])+i)
    if j + 1 == count_teor_sites[gene[0]]:
        chr_list = np.array(chr_list, dtype = np.int32)
        teor_sites[gene[0]] += chr_list
        del chr_list

with open(f'{path}/teor_A_T_sites.pickle', 'wb') as f:
    pickle.dump(teor_sites, f)

del teor_sites

with open(f'{path}/teor_A_T_sites.pickle', 'rb') as f:
    teor_sites = pickle.load(f)

### Вывод результатов в 24 различных файла для экономии оперативной памяти

In [27]:
k = 0
for j, gene in enumerate(genes):
    chr_list = []
    gene_by_letter = gene_search(gene[0], int(gene[1]), int(gene[2]))
    for i, letter in enumerate(gene_by_letter):
        if gene[3] == '+' and letter == 'A':
            chr_list.append(int(gene[1])+i)
        if gene[3] == '-' and letter == 'T':
            chr_list.append(int(gene[1])+i)
    
    if j - k + 1 == count_teor_sites[gene[0]]:
        print(len(chr_list))
        chr_list = np.array(chr_list, dtype = np.int32)
        with open(f'{path}/teor_A_T_sites_in_lists/teor_A_T_sites_{gene[0]}.pickle', 'wb') as f:
            pickle.dump(chr_list, f)
        print(gene[0])
        k = j
        del chr_list

43986698
chr1


NameError: name 'chr_list' is not defined

In [12]:
len(chr_list)

16517

### Распараллеливание

In [11]:
k = {x: y for x, y in zip(range(len(genes)), genes)}
k

{0: Interval(chr1:11868-14409),
 1: Interval(chr1:14403-29570),
 2: Interval(chr1:29553-31109),
 3: Interval(chr1:34553-36081),
 4: Interval(chr1:52472-53312),
 5: Interval(chr1:57597-64116),
 6: Interval(chr1:65418-71585),
 7: Interval(chr1:89294-133723),
 8: Interval(chr1:131024-134836),
 9: Interval(chr1:135140-135895),
 10: Interval(chr1:137681-137965),
 11: Interval(chr1:139789-140339),
 12: Interval(chr1:141473-173862),
 13: Interval(chr1:160445-161525),
 14: Interval(chr1:182695-184174),
 15: Interval(chr1:185216-195411),
 16: Interval(chr1:257863-359681),
 17: Interval(chr1:266854-268655),
 18: Interval(chr1:358856-366052),
 19: Interval(chr1:365388-522928),
 20: Interval(chr1:439869-440232),
 21: Interval(chr1:487100-489906),
 22: Interval(chr1:586070-827796),
 23: Interval(chr1:587628-594768),
 24: Interval(chr1:629061-629433),
 25: Interval(chr1:629639-630683),
 26: Interval(chr1:631073-632616),
 27: Interval(chr1:632756-633438),
 28: Interval(chr1:633534-634922),
 29: Inter

In [11]:
# Redefine, with only 1 mandatory argument.
def one_chrom_A_T_in_genes(j):
    """Функция принимает на вход индекс хромосомы (с нуля) и создает для нее список потенциальных сайтов редактирования по генам."""
    if j == 0:
        n = 0
        chr_list = [] 
    gene_by_letter = gene_search(k[j][0], int(k[j][1]), int(k[j][2]))
    for i, letter in enumerate(gene_by_letter):
        if k[j][3] == '+' and letter == 'A':
            chr_list.append(int(gene[1])+i)
        if k[j][3] == '-' and letter == 'T':
            chr_list.append(int(k[j][1])+i)
    if  j - n + 1 == count_teor_sites[k[j][0]]:
        chr_list = np.array(chr_list, dtype = np.int32)
        with open(f'{path}/teor_A_T_sites_in_lists/teor_A_T_sites_{k[j][0]}.pickle', 'wb') as f:
            pickle.dump(chr_list, f)
        print(k[j][0])
        n = j
        del chr_list

In [12]:
chr_list = []
pool = mp.Pool(mp.cpu_count())

results = pool.map(one_chrom_A_T_in_genes, [j for j in k])

pool.close()

NameError: name 'gene' is not defined

Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/matvey/anaconda3/envs/adar1/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/matvey/anaconda3/envs/adar1/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/matvey/anaconda3/envs/adar1/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/matvey/anaconda3/envs/adar1/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/matvey/anaconda3/envs/adar1/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/matvey/anaconda3/envs/adar1/lib/python3.8/multiprocessing/queues.py", line 355, in get
    with self._rlock:
  Fil

In [18]:
len(data_new)

142

### Распараллеливание 2

In [15]:
k = {x: y for x, y in zip(range(len(genes)), genes)}

In [ ]:
teor_sites = defaultdict(list)
def one_chrom_A_T_in_genes(j):
        if j + 1 == count_teor_sites[k[j][0]]:
            chr_list = []

        gene_by_letter = gene_search(k[j][0], int(k[j][1]), int(k[j][2]))
        for i, letter in enumerate(gene_by_letter):
            if k[j][3] == '+' and letter == 'A':
                chr_list.append(int(k[j][1])+i)
            if k[j][3] == '-' and letter == 'T':
                chr_list.append(int(k[j][1])+i)
        if j + 1 == count_teor_sites[k[j][0]]:
            chr_list = np.array(chr_list, dtype = np.int32)
            teor_sites[k[j][0]] += chr_list
            print(k[j])
            del chr_list
        else:
            

In [ ]:
try:
   a = int(input("Please enter a number: "))
   print(f'{a} squared is {a*a}')
except:
   print("Wrong input type! You must enter a number!")

In [13]:
for (j, gene) in enumerate("AAAA"):
    print(type((j, gene)))

<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


In [24]:
len(k)

49752

### Распараллеливание финал

In [12]:
count_teor_sites = defaultdict(int)
for chrom in genes:
    count_teor_sites[chrom.chrom] += 1

In [13]:
nule = 0
for chrom in count_teor_sites.keys():
    count_teor_sites[chrom] += nule
    nule = count_teor_sites[chrom]

In [14]:
count_teor_sites

defaultdict(int,
            {'chr1': 4624,
             'chr2': 8150,
             'chr3': 10717,
             'chr4': 12983,
             'chr5': 15468,
             'chr6': 18081,
             'chr7': 20563,
             'chr8': 22658,
             'chr9': 24607,
             'chrX': 26676,
             'chrY': 27181,
             'chr10': 29152,
             'chr11': 31909,
             'chr12': 34318,
             'chr13': 35467,
             'chr14': 37330,
             'chr15': 38950,
             'chr16': 40939,
             'chr17': 43306,
             'chr18': 44296,
             'chr19': 46685,
             'chr20': 47892,
             'chr21': 48620,
             'chr22': 49752})

In [15]:
nule = 0
n = dict()
for chrom in count_teor_sites.keys():
    n[chrom] = [nule, count_teor_sites[chrom]]
    nule = count_teor_sites[chrom]+1

In [16]:
n#первое число односится к текущей хромосоме. Второе - к следующей

{'chr1': [0, 4624],
 'chr2': [4625, 8150],
 'chr3': [8151, 10717],
 'chr4': [10718, 12983],
 'chr5': [12984, 15468],
 'chr6': [15469, 18081],
 'chr7': [18082, 20563],
 'chr8': [20564, 22658],
 'chr9': [22659, 24607],
 'chrX': [24608, 26676],
 'chrY': [26677, 27181],
 'chr10': [27182, 29152],
 'chr11': [29153, 31909],
 'chr12': [31910, 34318],
 'chr13': [34319, 35467],
 'chr14': [35468, 37330],
 'chr15': [37331, 38950],
 'chr16': [38951, 40939],
 'chr17': [40940, 43306],
 'chr18': [43307, 44296],
 'chr19': [44297, 46685],
 'chr20': [46686, 47892],
 'chr21': [47893, 48620],
 'chr22': [48621, 49752]}

In [30]:
def teor_esites_position(chrom):
    """Функция на вход принимает имя хромосомы. Возвращает список с вакантными координатами мест редактирования в межгеннике"""
    chr_list = []
    if chrom == 'chr1':
        for j, gene in enumerate(genes[n[chrom][0]: n[chrom][1]]):#при срезе бед файла берется [1;10) (невключительно справа). Но тк структура n учитывает это, то все нормально.Но так как структура n имеет свою специфику (смотреть описание к n), то приходится разбивать функцию на 2 части. 
            gene_by_letter = gene_search(chrom, int(gene[1]), int(gene[2]))
            matching_letter = 'T' if gene[3] == '-' else 'A'
            chr_sublist = (int(k[j][1])+i for i, letter in enumerate(gene_by_letter) if letter == matching_letter)
            chr_list.extend(chr_sublist)
    else:
        for j, gene in enumerate(genes[n[chrom][0]-1: n[chrom][1]]):#при срезе бед файла берется [1;10) (невключительно справа). Но тк структура n учитывает это, то все нормально.
            gene_by_letter = gene_search(chrom, int(gene[1]), int(gene[2]))
            matching_letter = 'T' if gene[3] == '-' else 'A'
            chr_sublist = (int(k[j][1])+i for i, letter in enumerate(gene_by_letter) if letter == matching_letter)
            #number_genes_in_previos_chrom = n['chr'+ chrom.split('r')[1]]
            chr_sublist = (int(k[n[chrom][0]-1 + j][1])+i for i, letter in enumerate(gene_by_letter) if letter == matching_letter)
            chr_list.extend(chr_sublist)
    chr_list = np.array(chr_list, dtype = np.int32)
    print(chrom)
    return chr_list

In [31]:
pool = mp.Pool(mp.cpu_count())
results = pool.map(teor_esites_position, {chrom: [] for chrom in n.keys()})
pool.close()

chr7
chr5
chr1
chr3
chr8
chr6
chrY
chr4
chr9
chr2
chr10
chr11
chrX
chr13
chr15
chr17
chr14
chr12
chr16
chr18
chr19
chr21
chr22
chr20


In [32]:
result_with_chrom_names = {chrom: [] for chrom in n.keys()}
for i, j in zip(result_with_chrom_names, results):
    result_with_chrom_names[i] = j

In [33]:
result_with_chrom_names

{'chr1': array([    11871,     11872,     11882, ..., 248937034, 248937035,
        248937041], dtype=int32),
 'chr2': array([    38813,     38814,     38817, ..., 242120024, 242120036,
        242120041], dtype=int32),
 'chr3': array([    11749,     11760,     11766, ..., 198228371, 198228372,
        198228374], dtype=int32),
 'chr4': array([    49095,     49096,     49097, ..., 190195962, 190195969,
        190195973], dtype=int32),
 'chr5': array([    58197,     58200,     58206, ..., 181472424, 181472428,
        181472429], dtype=int32),
 'chr6': array([    95123,     95126,     95128, ..., 170745969, 170745970,
        170745976], dtype=int32),
 'chr7': array([    12705,     12706,     12707, ..., 159233368, 159233372,
        159233376], dtype=int32),
 'chr8': array([    64092,     64109,     64115, ..., 145066657, 145066667,
        145066678], dtype=int32),
 'chr9': array([    12135,     12138,     12139, ..., 138320831, 138320833,
        138320834], dtype=int32),
 'chrX': a

In [43]:
with open(f'{path}/teor_sites_coordinats_in_genes.pickle', 'wb') as f:
    pickle.dump(result_with_chrom_names, f)

In [96]:
def teor_esites_position(chrom):
    chr_list = []
    for j, gene in enumerate(genes[n[chrom][0] : n[chrom][1]]):
        gene_by_letter = gene_search(chrom, int(gene[1]), int(gene[2]))
        for i, letter in enumerate(gene_by_letter):
            if gene[3] == '+' and letter == 'A':
                chr_list.append(int(k[j][1])+i)
            if gene[3] == '-' and letter == 'T':
                chr_list.append(int(k[j][1])+i)
    chr_list = np.array(chr_list, dtype = np.int32)
    print(chrom)
    return chr_list

In [92]:
def teor_esites_position(chrom):
    chr_list = []
    for j, gene in enumerate(genes[n[chrom][0]: n[chrom][1]]):
        gene_by_letter = gene_search(chrom, int(gene[1]), int(gene[2]))
        matching_letter = 'T' if gene[3] == '-' else 'A'
        chr_sublist = (int(k[j][1])+i for i, letter in enumerate(gene_by_letter) if letter == matching_letter)
        chr_list.extend(chr_sublist)
    chr_list = np.array(chr_list, dtype = np.int32)
    print(chrom)
    return chr_list